In [2]:
import json
import pandas as pd
from datasets import Dataset
from bs4 import BeautifulSoup
from tqdm import tqdm

In [16]:
data = []
with open('hf-datasets/raw-datasets/snapshot.jsonl') as fopen:
    for l in tqdm(fopen):
        data.append('\n'.join(json.loads(l)['text']))

373188it [00:07, 49879.22it/s]


In [17]:
len(data)

373188

In [18]:
data_dict = {
    'text': data
}

In [19]:
dataset = Dataset.from_dict(data_dict)

In [20]:
dataset

Dataset({
    features: ['text'],
    num_rows: 373188
})

In [21]:
dataset.save_to_disk("hf-datasets/raw-datasets/snapshot")

Saving the dataset (0/5 shards):   0%|          | 0/373188 [00:00<?, ? examples/s]

In [22]:
!du -hs hf-datasets/raw-datasets/snapshot

2.0G	hf-datasets/raw-datasets/snapshot


### 2. Run below command

make sure directory for the script is correct before running the command

- path -  your dataset to dedupe path
- name - relevant only when u use dataset directly from huggingface
- output - output directory of the dedupe dataset
- column - column of the text we are going to remove duplicate
- threshold - jaccard similarity threshold
- local - (local dataset or directly from huggingface) remove if dataset is not local

In [23]:
command = f"python3 -m text_dedup.minhash \
  --path hf-datasets/raw-datasets/snapshot \
  --split train \
  --cache_dir ./cache \
  --output hf-datasets/dedupe-datasets/snapshot \
  --column text \
  --batch_size 10000 \
  --threshold 0.95 \
  --min_length 1 \
  --local"

command

'python3 -m text_dedup.minhash   --path hf-datasets/raw-datasets/snapshot   --split train   --cache_dir ./cache   --output hf-datasets/dedupe-datasets/snapshot   --column text   --batch_size 10000   --threshold 0.95   --min_length 1   --local'

In [25]:
!rm -rf hf-datasets/dedupe-datasets/snapshot

In [30]:
import subprocess
subprocess.run(command, shell=True)

The script produced the following

- duplicates.csv : containing duplicate pairs indices (document index)
- duplicate_cluster.csv: the text of duplicate pairs for view
- the deduped dataset in huggingface dataset type (from output argument)
- dedup dataset in jsonl format

In [27]:
!wc -l hf-datasets/dedupe-datasets/snapshot.jsonl

209177 hf-datasets/dedupe-datasets/snapshot.jsonl


In [28]:
!ls -lh hf-datasets/dedupe-datasets/snapshot.jsonl

-rw-r--r-- 1 ubuntu ubuntu 1.3G Aug 18 02:43 hf-datasets/dedupe-datasets/snapshot.jsonl


In [29]:
!rm -rf hf-datasets/dedupe-datasets/snapshot hf-datasets/raw-datasets/snapshot